In [ ]:
import time
import torch
from torch.utils.data import TensorDataset, DataLoader

import numpy as np

import sys
sys.path.append("../..") 
import kinematics.allegro_hand as allegro
import tools.rotations as rot
from nn_model import Net, loss_SCA, DataLoaderX

%load_ext autoreload
%autoreload 2

In [ ]:
hand = allegro.Robot(use_fingers=[1,1,1,1], path_prefix='../../', all_link_fk=True, meshes=True)
lb = hand.joint_lb
ub = hand.joint_ub




In [ ]:
hand_joint_bound = np.concatenate([lb.reshape(1,-1), ub.reshape(1,-1)], axis=0)
hand_joint_bound

In [ ]:
np.savetxt('models/hand_joint_bound.txt', hand_joint_bound, delimiter=' ')

In [ ]:
hand_joint_bounds = np.loadtxt('models/hand_joint_bound.txt')

In [ ]:
hand_joint_bounds

In [ ]:
# load data
path = '../dataset/'
dataset_name = 'dataset'  
nums = 10000000
suffix = '_15dis_sphere_2'
# dataset_name = 'test100000.txt'  # 59min 100k nums
# data2 = np.loadtxt(path+dataset_name, delimiter=' ')  # 
data2 = np.load(path + dataset_name + str(nums) + suffix + '.npy')   #  # 16 joints, 3 obj pose, 15 min_dis
print('data2 shape:', data2.shape)
nums = 1000000
data2 = data2[:nums,:]


dis_with_obj = list(np.array([5,9,12,14,15]) + 18)
dis_with_obj +=[16,17,18]
# dis_with_obj
dis_only_hand = []
for i in range(34):
    if i not in dis_with_obj:
        dis_only_hand.append(i)
print(dis_only_hand)
data1 = data2[:, dis_only_hand] # 16 joints, 10 min_dis

In [ ]:
print('nonzero:',np.count_nonzero(np.min(data1[:, 16:], axis=1 ))/nums)

In [ ]:
max_dis = [np.max(data1[:, i] ) for i in range(16,26)]
max_dis = np.asarray(max_dis)
max_dis_ = np.max(max_dis)

In [ ]:
min_dis = np.min(max_dis)
min_dis

In [ ]:
# normalize data to [-1,1]

data = np.copy(data1)
for i in range(16):  # joint angles
    data[:,i] = (data2[:,i] - lb[i] )/ (ub[i] - lb[i]) *2 -1

data_dis = data1[:,16:]/max_dis  # normalize for each dis of 10 
data_dis[data_dis==0] = -1

data[:, 16:] = data_dis



add_sin_cos = True
keep_all_dis = True

num = int(data.shape[0] * 0.8)
batch_size = 128 * 64 


if add_sin_cos:
    num_s = 16 * 3
    if keep_all_dis:
        data = np.concatenate([data[:,:16], np.sin(data[:,:16]), np.cos(data[:,:16]), data[:,16:]], axis=1)
    else:
        min_dis = np.min(data1[:, 16:], axis=1 ).reshape(-1, 1)
        data = np.concatenate([data[:,:16], np.sin(data[:,:16]), np.cos(data[:,:16]), min_dis], axis=1)
else:
    num_s = 16
    if keep_all_dis is False:
        min_dis = np.min(data1[:, 16:], axis=1 ).reshape(-1, 1)
        data = np.concatenate([data[:,:16], min_dis], axis=1)


x_train = torch.Tensor(data[:num, :num_s])
y_train = torch.Tensor(data[:num, num_s:])
dim_in = x_train.size(1)
dim_out = y_train.size(1)
dataset_train = TensorDataset(x_train,y_train) # create your datset
loader_train = DataLoaderX(dataset_train,batch_size=batch_size, num_workers=16, pin_memory=True,prefetch_factor=4, persistent_workers=True)
x_test = torch.Tensor(data[num:, :num_s])
y_test = torch.Tensor(data[num:, num_s:])

print(dim_in, dim_out)
del data2
del data1
del data_dis


In [ ]:
x_train

In [ ]:
data[:num, :num_s].nbytes/1024/1024 # MB for memory

In [ ]:
print(sys.getsizeof(torch.FloatTensor([0.5])))

In [ ]:
# sys.getsizeof(x_train)
sys.getsizeof(x_train.storage())/1024/1024 # MB for memory

In [ ]:
use_cuda = True
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

In [ ]:
import torch.nn as nn


In [ ]:
nums = 10000000
suffix = '_15dis_sphere_2'
use_cuda = True
net = Net(suffix, nums, use_cuda=True, batch_size=20000)
net.to(net.device)


print(net)
params = list(net.parameters())
# print(params)
print(len(params))
# print(params[0].size())

In [ ]:
# def weights_init(m):                                               
#     nn.init.normal_(m.weight.data, 0.0, 0.02) 

In [ ]:
# for m in net.children():
#     if isinstance(m, nn.Linear):
#         nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#         nn.init.constant_(m.bias, 0)



In [ ]:
# for i in net.parameters():
#     print(i)

In [ ]:
weight_p, bias_p = [],[]
for name, p in net.named_parameters():
    if 'bias' in name:
        bias_p += [p]
    else:
        weight_p += [p]



In [ ]:
from IPython.display import clear_output
# %% Loss fcn
import torch.optim as optim

criterion = nn.MSELoss()
# optimizer = optim.SGD(net.parameters(),lr=0.01)
# optimizer = optim.Adam(net.parameters(),lr=0.01,weight_decay=0.001)
optimizer = optim.SGD([
          {'params': weight_p, 'weight_decay':1e-5},
          {'params': bias_p, 'weight_decay':0}
          ], lr=15e-2, momentum=0.9)


# net.to(device)


t_all = []
error_all = []

t_start = time.time()
i = 0
while True:
    t0 = time.time()
    
    for batch_idx, (x, y) in enumerate(net.loader_train):
        optimizer.zero_grad() # zero the parameter gradients    
        if use_cuda:
            x = x.cuda()
            y = y.cuda()
        
        # forward + backward + optimize
        outputs = net(x)
        loss = loss_SCA(outputs, y) #y[0,0,2,3] out[0,-0.1,3,4]
        # loss= criterion(outputs, y)
        
        loss.backward()
        optimizer.step()   
    
    # del x
    # del y
    # error_all.append(np.sqrt(loss.data.item())*max_dis_)
    if(i%5==0):
        if i%50==0:
            clear_output(wait=True)
        print(i,"%0.4f"%(time.time() - t0), np.sqrt(loss.data.item()) * net.max_dis_ )
    
    t1 = time.time() - t0
    t_all.append(t1)
    i +=1
    # if time.time() - t_start>3600*4:
    #     break
    if i>100:
        break
t_all = np.asarray(t_all)
print('each step', np.mean(t_all))  


In [ ]:
net.validate(net)

In [ ]:
import numpy as np
np.random.randint(0,100)

In [ ]:
np.sqrt(loss.data.item())  * max_dis 

In [ ]:
net.train()

In [ ]:
loss

In [ ]:
max_dis

In [ ]:
net.x_test_gpu

In [ ]:
# verification


with torch.no_grad():
    outputs = net(net.x_test_gpu)

outputs = outputs.cpu().numpy()

outputs
# error = (data[num:, num_s:] - outputs)
# np.sqrt(np.mean(error*error))  * max_dis




In [ ]:
# in training dataset
x_test_1 = x_test.to(device)
y_test_1 = y_test.to(device)
net.to(device)

with torch.no_grad():
    outputs = net(x_gpu)

outputs = outputs.cpu().numpy()

error = (data[:num, num_s:] - outputs)
np.sqrt(np.mean(error*error)) * max_dis




In [ ]:
# save the model
filepath = 'models/linear4_tanh_4_4hours'
torch.save(net.state_dict(), filepath)

